In [11]:
!pip install groq  # Install the groq module

from groq import Groq
from IPython.display import HTML, display
from ipywidgets import Text, Button, Output, VBox, HBox

from google.colab import userdata
userdata.get('first')

client = Groq(api_key=userdata.get('first'))

preprompt = "give me academic level very short and to the point answer no more than 1 sentence"
conversation_history = []

def get_response(model_name, message):
    global conversation_history

    # Add preprompt to the user message before adding to history
    preprompted_message = preprompt + " " + message
    conversation_history.append({"role": "user", "content": preprompted_message})

    completion = client.chat.completions.create(
        model=model_name,
        messages=conversation_history,
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    response = ""
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""

    conversation_history.append({"role": "assistant", "content": response})
    return response

# Create widgets for input, output, and clear button
input_box = Text(description='Your Message:', placeholder='Type your message here...', layout={'width': '90%'})
input_box.style.description_width = 'initial'
submit_button = Button(description='Submit', button_style='success', layout={'width': '100px'})
clear_button = Button(description='Clear', button_style='warning', layout={'width': '100px'})
output_area = Output()

# Function to handle button click and enter key press
def on_submit(b):
    message = input_box.value
    if message:  # Send only if the message is not empty
        llama3_response = get_response("llama3-70b-8192", message)
        html = f"""
        <div class="message-container" style="border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; border-radius: 5px;">
            <h2 style="margin-bottom: 5px;">Message:</h2>
            <div class="message-content">
                <div class="message-text">{message}</div>
            </div>
        </div>
        <div class="response-container" style="border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; border-radius: 5px;">
            <h2 style="margin-bottom: 5px;">Response:</h2>
            <div class="response-content">
                <div class="response-text">{llama3_response}</div>
            </div>
        </div>
        """
        with output_area:
            display(HTML(html))
        input_box.value = ''  # Clear the input box

# Function to clear the output area and conversation history
def on_clear_clicked(b):
    global conversation_history
    conversation_history = []
    output_area.clear_output()

# Link button clicks and enter key press to the functions
submit_button.on_click(on_submit)
clear_button.on_click(on_clear_clicked)  # Fixed the typo 'clear button' -> 'clear_button'
input_box.on_submit(on_submit)  # Handle enter key press

# Display widgets in the notebook
buttons_box = HBox([submit_button, clear_button], layout={'justify_content': 'center', 'padding': '10px'})
display(VBox([input_box, buttons_box, output_area]))